In [161]:
# imports
import requests
import pandas as pd
import os

#EBIKES DATAFRAME

In [162]:
url = "http://api.citybik.es/"
href = "/v2/networks/"
#network_id = 'velib'
response = requests.get(url + href)
# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary
    data = response.json()
else:
    print("Request failed with status code:", response.status_code)
networks = data['networks']
city = 'Vancouver'
filtered_network = [network for network in networks if network['location']['city'] == city]
network_id = filtered_network[0]['href']
response1 = requests.get(url + network_id)
if response1.status_code == 200:
    # Parse the JSON response into a Python dictionary
    data1 = response1.json()
else:
    print("Request failed with status code:", response.status_code)
vancouver_network_stations = pd.json_normalize(data=data1, record_path=['network', 'stations'])
vancouver_network_stations = vancouver_network_stations[['free_bikes', 'id', 'longitude', 'latitude']]
vancouver_network_stations

,free_bikes,id,longitude,latitude
0,8,7a19c49f486d7c0c02b3685d7b240448,-123.114397,49.262487
1,11,32603a87cfca71d0f7dfa3513bad69d5,-123.121817,49.274566
2,0,6d42fa40360f9a6b2bf641c7b8bb2862,-123.110154,49.279764
3,8,66f873d641d448bd1572ab086665a458,-123.113504,49.260599
4,0,485d4d24c803cfde829ab89699fed833,-123.117772,49.264215
...,...,...,...,...
247,7,10acadd77bc21f26a7f09d8c74d4af2d,-123.114237,49.277773
248,1,58345bb719f4e570ed4edc9dfa6a68a2,-123.254307,49.263073
249,5,261efc67071963969b3f848c66454d97,-123.250902,49.259999
250,4,ee620d77724c8993b0d366e7ec655b64,-123.236210,49.254780


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [163]:
#Testing to see if get request works
radius_meters = 1000
url = url = "https://api.foursquare.com/v3/places/search"
four_square_api_key = os.environ['FOURSQUARE_API_KEY']
headers = {'Accept': 'application/json'}
headers['Authorization'] = four_square_api_key
params = {
    'll': "49.262487,-123.114397",
    'fields': 'fsq_id,name,location,categories,distance,description,price,rating',
    'radius': radius_meters
}

result = requests.get(url, headers=headers, params=params)

# Check if the request was successful (status code 200)
if result.status_code == 200:
# Parse the JSON response into a Python dictionary
    fsq_json = result.json()
    print(fsq_json)
else:
    print("Request failed with status code:", result.status_code)


{'results': [{'fsq_id': '4e223f06d4c0d32590f80ff4', 'categories': [{'id': 13306, 'name': 'Taco Restaurant', 'short_name': 'Tacos', 'plural_name': 'Taco Restaurants', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/taco_', 'suffix': '.png'}}], 'distance': 164, 'location': {'address': '2450 Yukon St', 'address_extended': 'at W Broadway', 'country': 'CA', 'cross_street': 'at W Broadway', 'formatted_address': '2450 Yukon St (at W Broadway), Vancouver BC V5Y 0A4', 'locality': 'Vancouver', 'postcode': 'V5Y 0A4', 'region': 'BC'}, 'name': 'La Taqueria Pinche Taco Shop', 'price': 2, 'rating': 8.4}, {'fsq_id': '4aa73d25f964a520704c20e3', 'categories': [{'id': 17069, 'name': 'Grocery Store', 'short_name': 'Grocery Store', 'plural_name': 'Grocery Stores', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_', 'suffix': '.png'}}, {'id': 17071, 'name': 'Health Food Store', 'short_name': 'Health Food Store', 'plural_name': 'Health Food Stores', 'icon': {'pref

In [164]:
df = pd.DataFrame(fsq_json['results'])
df.head()

,fsq_id,categories,distance,location,name,price,rating,description
0,4e223f06d4c0d32590f80ff4,"[{'id': 13306, 'name': 'Taco Restaurant', 'sho...",164,"{'address': '2450 Yukon St', 'address_extended...",La Taqueria Pinche Taco Shop,2.0,8.4,NaN
1,4aa73d25f964a520704c20e3,"[{'id': 17069, 'name': 'Grocery Store', 'short...",182,"{'address': '510 8th Ave W', 'country': 'CA', ...",Whole Foods,3.0,8.6,"Welcome to Vancouver, BC Whole Foods Market! W..."
2,588e7f618b0ebb5d010abac4,"[{'id': 13272, 'name': 'Ramen Restaurant', 'sh...",186,"{'address': '558 Broadway W', 'country': 'CA',...",Hokkaido Ramen Santouka,NaN,7.8,NaN
3,4aa843e0f964a5207f5020e3,"[{'id': 13068, 'name': 'American Restaurant', ...",232,"{'address': '575 West Broadway', 'country': 'C...",Cactus Club Cafe Broadway + Ash,3.0,7.5,Cactus Club Cafe is the leading casual fine di...
4,4ab145f3f964a520d56820e3,"[{'id': 13002, 'name': 'Bakery', 'short_name':...",283,"{'address': '368 7th Ave W', 'country': 'CA', ...",Solly's Bagelry,2.0,7.9,NaN


In [168]:
fsq_df = pd.DataFrame()
radius_meters = 1000
url = url = "https://api.foursquare.com/v3/places/search"
four_square_api_key = os.environ['FOURSQUARE_API_KEY']
headers = {'Accept': 'application/json'}
headers['Authorization'] = four_square_api_key

#itterate over all the ebikes stations above using longitude and latitude to get POI
for index, row in vancouver_network_stations.iterrows():
    params = {
    'll': f"{row['latitude']},{row['longitude']}",
    'fields': 'fsq_id,name,location,categories,distance,description,price,rating',
    'radius': radius_meters
}
    result = requests.get(url, headers=headers, params=params)

# Check if the request was successful (status code 200)
    if result.status_code == 200:
    # Parse the JSON response into a Python dictionary
        fsq_json = result.json()
        df = pd.DataFrame(fsq_json['results'])
        df['station_id'] = row['id']
        fsq_df = pd.concat([fsq_df, df], ignore_index=True)
#else:
    print("Request failed with status code:", result.status_code)

Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
R

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [169]:
fsq_df.head()

,fsq_id,categories,distance,location,name,price,rating,description,station_id
0,4e223f06d4c0d32590f80ff4,"[{'id': 13306, 'name': 'Taco Restaurant', 'sho...",164,"{'address': '2450 Yukon St', 'address_extended...",La Taqueria Pinche Taco Shop,2.0,8.4,NaN,7a19c49f486d7c0c02b3685d7b240448
1,4aa73d25f964a520704c20e3,"[{'id': 17069, 'name': 'Grocery Store', 'short...",182,"{'address': '510 8th Ave W', 'country': 'CA', ...",Whole Foods,3.0,8.6,"Welcome to Vancouver, BC Whole Foods Market! W...",7a19c49f486d7c0c02b3685d7b240448
2,588e7f618b0ebb5d010abac4,"[{'id': 13272, 'name': 'Ramen Restaurant', 'sh...",186,"{'address': '558 Broadway W', 'country': 'CA',...",Hokkaido Ramen Santouka,NaN,7.8,NaN,7a19c49f486d7c0c02b3685d7b240448
3,4aa843e0f964a5207f5020e3,"[{'id': 13068, 'name': 'American Restaurant', ...",232,"{'address': '575 West Broadway', 'country': 'C...",Cactus Club Cafe Broadway + Ash,3.0,7.5,Cactus Club Cafe is the leading casual fine di...,7a19c49f486d7c0c02b3685d7b240448
4,4ab145f3f964a520d56820e3,"[{'id': 13002, 'name': 'Bakery', 'short_name':...",283,"{'address': '368 7th Ave W', 'country': 'CA', ...",Solly's Bagelry,2.0,7.9,NaN,7a19c49f486d7c0c02b3685d7b240448


In [170]:
fsq_df['categories']

def extract_values(dictionary_list):
    values = [list(d.values())[1] for d in dictionary_list if d][0]  # Extracting values from dictionaries
    return values
fsq_df['category'] = fsq_df['categories'].apply(lambda x: extract_values(x))
fsq_parsed = fsq_df[['station_id', 'name', 'category', 'distance', 'rating', 'price']]
fsq_parsed

,station_id,name,category,distance,rating,price
0,7a19c49f486d7c0c02b3685d7b240448,La Taqueria Pinche Taco Shop,Taco Restaurant,164,8.4,2.0
1,7a19c49f486d7c0c02b3685d7b240448,Whole Foods,Grocery Store,182,8.6,3.0
2,7a19c49f486d7c0c02b3685d7b240448,Hokkaido Ramen Santouka,Ramen Restaurant,186,7.8,NaN
3,7a19c49f486d7c0c02b3685d7b240448,Cactus Club Cafe Broadway + Ash,American Restaurant,232,7.5,3.0
4,7a19c49f486d7c0c02b3685d7b240448,Solly's Bagelry,Bakery,283,7.9,2.0
...,...,...,...,...,...,...
2509,e02dd224b0177c65418bcb3f72d2f77d,Donald's Market,Grocery Store,501,8.7,NaN
2510,e02dd224b0177c65418bcb3f72d2f77d,Tacofino Commissary,Mexican Restaurant,506,8.6,2.0
2511,e02dd224b0177c65418bcb3f72d2f77d,Platform 7 Coffee,Café,507,8.4,1.0
2512,e02dd224b0177c65418bcb3f72d2f77d,The Downlow Chicken Shack,Fried Chicken Joint,610,8.6,1.0


Put your parsed results into a DataFrame

In [171]:
fsq_parsed.head()

,station_id,name,category,distance,rating,price
0,7a19c49f486d7c0c02b3685d7b240448,La Taqueria Pinche Taco Shop,Taco Restaurant,164,8.4,2.0
1,7a19c49f486d7c0c02b3685d7b240448,Whole Foods,Grocery Store,182,8.6,3.0
2,7a19c49f486d7c0c02b3685d7b240448,Hokkaido Ramen Santouka,Ramen Restaurant,186,7.8,NaN
3,7a19c49f486d7c0c02b3685d7b240448,Cactus Club Cafe Broadway + Ash,American Restaurant,232,7.5,3.0
4,7a19c49f486d7c0c02b3685d7b240448,Solly's Bagelry,Bakery,283,7.9,2.0


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

PLEASE NOTE THAT THE BELOW GET REQUESTS WORKED SEVERAL DAYS AGO AND URL, PARAMERTERS, HEADERS WORK IN POSTMAN, BUT THE GET REQUEST STILL RETURNS AN ERROR. I SPENT OVER ONE HOUR WITH LARRY AI AND COULDN'T SOLVE THE PROBLEM AND OVER ONE HOUR WITH A MENTOR WHO COULDN'T SOLVE THE PROBLEM. I TRIED SEVERAL ACCOUNTS IN YELP AND I AM UNABLE TO SPEND MORE TIME TRYING TO SOLVE THIS SO I AM GOING TO USE TWO SEPERATE JSON FILES COPIED FROM POSTMAN TO MIMICK GET REQUESTS TO PROVE I CAN MANIPULATE AND DOWNLOAD INFORMATION TO A PANDAS DATAFRAME


In [181]:
import urllib.request
import urllib.parse
import json

yelp_df = pd.DataFrame()
api_key = os.environ['YELP_API_KEY']
url = "https://api.yelp.com/v3/businesses/search"

headers = {
    "Authorization": f"Bearer {api_key}"
}
    
for index, row in vancouver_network_stations.iterrows():
    params = {
        "latitude": row['latitude'],
        "longitude": row['longitude'],
        "term": "restaurants",
        "radius": 1000,
        "limit": 10
    }
    query_string = urllib.parse.urlencode(params)
    url_with_params = f"{url}?{query_string}"
    req = urllib.request.Request(url_with_params, headers=headers)

    try:
        with urllib.request.urlopen(req) as response:
            response_body = response.read()
            data = json.loads(response_body.decode("utf-8"))
            df = pd.DataFrame(data['businesses'])
            df['station_id'] = row['id']
            yelp_df = pd.concat([yelp_df, df], ignore_index=True) 
    except urllib.error.HTTPError as e:
        print(f"HTTP error occurred: {e.code} - {e.reason}")
    except urllib.error.URLError as e:
        print(f"URL error occurred: {e.reason}")
    


In [182]:
yelp_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,attributes,price,station_id
0,F5wxgIiZE7LYQxgqhI483A,seaport-city-seafood-restaurant-vancouver,Seaport City Seafood Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/zRW3aV...,False,https://www.yelp.com/biz/seaport-city-seafood-...,52,"[{'alias': 'dimsum', 'title': 'Dim Sum'}, {'al...",4.3,"{'latitude': 49.263921119674905, 'longitude': ...",[],"{'address1': '2425 Cambie Street', 'address2':...",+16042227288,+1 604-222-7288,168.615262,"{'business_temp_closed': None, 'menu_url': Non...",NaN,7a19c49f486d7c0c02b3685d7b240448
1,XAH2HpuUUtu7CUO26pbs4w,saku-vancouver-8,Saku,https://s3-media2.fl.yelpcdn.com/bphoto/RJV1GX...,False,https://www.yelp.com/biz/saku-vancouver-8?adju...,250,"[{'alias': 'japanese', 'title': 'Japanese'}]",4.3,"{'latitude': 49.2631006, 'longitude': -123.116...",[],"{'address1': '548 W Broadway', 'address2': '',...",+16044234600,+1 604-423-4600,178.845344,"{'business_temp_closed': None, 'menu_url': 'ht...",$$,7a19c49f486d7c0c02b3685d7b240448
2,NensKn1MSVU_rm-1Y6WlFA,marulilu-cafe-vancouver,Marulilu Cafe,https://s3-media3.fl.yelpcdn.com/bphoto/jUmSJk...,False,https://www.yelp.com/biz/marulilu-cafe-vancouv...,290,"[{'alias': 'cafes', 'title': 'Cafes'}]",4.0,"{'latitude': 49.26338, 'longitude': -123.11415}",[],"{'address1': '451 W Broadway', 'address2': '',...",+16045684211,+1 604-568-4211,102.329817,"{'business_temp_closed': None, 'menu_url': 'ht...",$$,7a19c49f486d7c0c02b3685d7b240448
3,6iOAgzJ0DRZNSKA3FSrrOg,la-taqueria-pinche-taco-shop-vancouver,La Taqueria Pinche Taco Shop,https://s3-media1.fl.yelpcdn.com/bphoto/fjhIj3...,False,https://www.yelp.com/biz/la-taqueria-pinche-ta...,685,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.2,"{'latitude': 49.263559, 'longitude': -123.112736}",[],"{'address1': '2450 Yukon Street', 'address2': ...",+16045582549,+1 604-558-2549,169.517456,"{'business_temp_closed': None, 'menu_url': '',...",$$,7a19c49f486d7c0c02b3685d7b240448
4,y583PSvm5c6tZYmRiboYYA,a-bento-vancouver,A.Bento,https://s3-media4.fl.yelpcdn.com/bphoto/2Ger31...,False,https://www.yelp.com/biz/a-bento-vancouver?adj...,32,"[{'alias': 'taiwanese', 'title': 'Taiwanese'},...",3.9,"{'latitude': 49.263164, 'longitude': -123.118857}",[],"{'address1': '656W Broadway', 'address2': None...",+16044239585,+1 604-423-9585,332.277954,"{'business_temp_closed': None, 'menu_url': Non...",$$,7a19c49f486d7c0c02b3685d7b240448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2502,maVufqgNOkyXx31MjiQA0A,jackalopes-neighbourhood-dive-vancouver,Jackalope's Neighbourhood Dive,https://s3-media4.fl.yelpcdn.com/bphoto/2v2bDp...,False,https://www.yelp.com/biz/jackalopes-neighbourh...,123,"[{'alias': 'divebars', 'title': 'Dive Bars'}, ...",4.3,"{'latitude': 49.2812849, 'longitude': -123.058...",[],"{'address1': '2257 E Hastings Street', 'addres...",+16045686674,+1 604-568-6674,456.399411,"{'business_temp_closed': None, 'menu_url': Non...",$$,e02dd224b0177c65418bcb3f72d2f77d
2503,BsaN_ft3Rh75aEGIcJTtIA,via-tevere-pizzeria-vancouver,Via Tevere Pizzeria,https://s3-media3.fl.yelpcdn.com/bphoto/EI2fVY...,False,https://www.yelp.com/biz/via-tevere-pizzeria-v...,337,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.1,"{'latitude': 49.27387530438791, 'longitude': -...",[],"{'address1': '1190 Victoria Drive', 'address2'...",+16043361803,+1 604-336-1803,502.045245,"{'business_temp_closed': None, 'menu_url': 'ht...",$$,e02dd224b0177c65418bcb3f72d2f77d
2504,Laiv2EnIwZFMF9Jy-R0PAA,absinthe-bistro-vancouver,Absinthe Bistro,https://s3-media2.fl.yelpcdn.com/bphoto/F3pclo...,False,https://www.yelp.com/biz/absinthe-bistro-vanco...,142,"[{'alias': 'french', 'title': 'French'}, {'ali...",4.6,"{'latitude': 49.27607, 'longitude': -123.0692457}",[],"{'address1': '952 Commercial Drive', 'address2...",+16045669053,+1 604-566-9053,589.228697,"{'business_temp_closed': None, 'menu_url': 'ht...",$$$,e02dd224b0177

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [184]:
yelp_df['category'] = df['categories'].apply(lambda x: [a['alias'] for a in x][0])
yelp_df['category'] = yelp_df['category']
yelp_parsed = yelp_df[['station_id', 'name', 'category', 'distance', 'rating', 'price']]
yelp_parsed

,station_id,name,category,distance,rating,price
0,7a19c49f486d7c0c02b3685d7b240448,Seaport City Seafood Restaurant,vietnamese,168.615262,4.3,NaN
1,7a19c49f486d7c0c02b3685d7b240448,Saku,italian,178.845344,4.3,$$
2,7a19c49f486d7c0c02b3685d7b240448,Marulilu Cafe,diners,102.329817,4.0,$$
3,7a19c49f486d7c0c02b3685d7b240448,La Taqueria Pinche Taco Shop,wine_bars,169.517456,4.2,$$
4,7a19c49f486d7c0c02b3685d7b240448,A.Bento,thai,332.277954,3.9,$$
...,...,...,...,...,...,...
2502,e02dd224b0177c65418bcb3f72d2f77d,Jackalope's Neighbourhood Dive,NaN,456.399411,4.3,$$
2503,e02dd224b0177c65418bcb3f72d2f77d,Via Tevere Pizzeria,NaN,502.045245,4.1,$$
2504,e02dd224b0177c65418bcb3f72d2f77d,Absinthe Bistro,NaN,589.228697,4.6,$$$
2505,e02dd224b0177c65418bcb3f72d2f77d,Caffe La Tana,NaN,680.684474,4.6,NaN


Put your parsed results into a DataFrame

In [185]:
yelp_parsed

,station_id,name,category,distance,rating,price
0,7a19c49f486d7c0c02b3685d7b240448,Seaport City Seafood Restaurant,vietnamese,168.615262,4.3,NaN
1,7a19c49f486d7c0c02b3685d7b240448,Saku,italian,178.845344,4.3,$$
2,7a19c49f486d7c0c02b3685d7b240448,Marulilu Cafe,diners,102.329817,4.0,$$
3,7a19c49f486d7c0c02b3685d7b240448,La Taqueria Pinche Taco Shop,wine_bars,169.517456,4.2,$$
4,7a19c49f486d7c0c02b3685d7b240448,A.Bento,thai,332.277954,3.9,$$
...,...,...,...,...,...,...
2502,e02dd224b0177c65418bcb3f72d2f77d,Jackalope's Neighbourhood Dive,NaN,456.399411,4.3,$$
2503,e02dd224b0177c65418bcb3f72d2f77d,Via Tevere Pizzeria,NaN,502.045245,4.1,$$
2504,e02dd224b0177c65418bcb3f72d2f77d,Absinthe Bistro,NaN,589.228697,4.6,$$$
2505,e02dd224b0177c65418bcb3f72d2f77d,Caffe La Tana,NaN,680.684474,4.6,NaN


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp provides more complete data: in addtion to foursquare location data yelp provides: reviews
Yelp is catered to restaurants, but Foursquare has other points of interest as well. If you are looking for details about restaurants Yelp is bettter, but for other POIs Foursquare has better coverage.
Yelp get requests also gives many results while Foursquare only returns 10 results per request
Foursquare gives you $200 credit per month - I used $20 for the project
Yelp gives 300 requests per day

Get the top 10 restaurants according to their rating

In [187]:
#foursquare top 10
fsq_filtered = fsq_parsed[fsq_parsed['category'].str.contains('Restaurant')]
fsq_sorted = fsq_filtered.sort_values(by='rating', ascending=False)
top_10 = fsq_sorted.drop_duplicates(subset=['name'])
top_10.head(10)

,station_id,name,category,distance,rating,price
841,c448b73d92f7f8e1daafb3b9c5fbe6a6,The Mackenzie Room,Restaurant,100,9.2,NaN
1700,ce5dccf5a0f190bb4567a2fd93e7971e,Kingyo Izakaya 金魚居酒屋,Sushi Restaurant,56,9.1,3.0
90,6993b3dbb0758927967592ea612a2b1e,Hawksworth Restaurant,Restaurant,125,9.1,4.0
1548,5a34495280727a5b50b9aa41e14e9a16,Le Crocodile Restaurant,French Restaurant,349,9.1,4.0
1864,5de0fe55497681d4cc9dd25509cf51ab,Ramen Danbo - Robson,Ramen Restaurant,332,9.1,2.0
1894,9c1397c06e07bf74fc049f1a0872eea0,Green Leaf Sushi,Sushi Restaurant,448,9.0,2.0
1609,257715b845828e027ff35dd0f11c971e,Miku,Sushi Restaurant,250,9.0,3.0
505,97b25faf822b93fdbf72da1d594c0288,MeeT in Gastown,Vegan and Vegetarian Restaurant,239,9.0,2.0
1818,0e0aef6fabaa808b3a3cef24657257a4,Tacofino,Taco Restaurant,314,9.0,1.0
1643,d085c6749a32517e1fb0d070e4128402,Cactus Club Cafe,Mexican Restaurant,347,9.0,2.0


In [191]:
#Yelp top 10
yelp_parsed = yelp_df[['station_id', 'name', 'category', 'distance', 'review_count', 'rating', 'price']]
yelp_sorted = yelp_parsed[yelp_parsed['review_count'] > 1000].sort_values(by='rating', ascending=False)
top_10 = yelp_sorted.drop_duplicates(subset=['name'])
top_10.head(10)

,station_id,name,category,distance,review_count,rating,price
507,97b25faf822b93fdbf72da1d594c0288,Miku,NaN,894.207539,2295,4.4,$$$
480,efa947b901a2cf85b58320b3ad26a2fe,Kingyo,NaN,300.703550,1142,4.4,$$
441,1d4153659a0a0a2089078c739df6737d,Blue Water Cafe,NaN,181.156262,1100,4.4,$$$$
329,c7b5bec0b2f4ab9a2d3f9d837725620c,Jam Cafe on Beatty,NaN,578.873405,1432,4.4,$$
550,e3d32ad772a5898c3b96c4d23d2cd542,Guu with Garlic,NaN,1324.691115,1067,4.3,$$
1039,1648ead5ea617d2cea115869d9516f5e,Medina Cafe,NaN,890.652258,2625,4.2,$$
1035,1648ead5ea617d2cea115869d9516f5e,Phnom Penh,NaN,649.259504,1677,4.2,$$
1612,910027847876403b57f1013c16504688,Chambar,NaN,559.597027,1544,4.2,$$
29,6d42fa40360f9a6b2bf641c7b8bb2862,Twisted Fork,NaN,562.988187,1116,4.1,$$
2317,cc2084e365f6f230fe26bae0b6867775,The Flying Pig,NaN,624.637418,1326,4.0,$$


PUTTING THE INDIVIDUAL DATAFRAMES INTO A DATABASE SINCE MY DAILY API LIMIT IS LIMITED

In [173]:
from sqlalchemy import create_engine
sqlite_db = 'sqlite:///../data/ebikes.db'
engine = create_engine(sqlite_db)

In [192]:

fsq_parsed.to_sql('foursquare', con=engine, index=False)
yelp_parsed.to_sql('yelp', con=engine, index=False)
vancouver_network_stations.to_sql('ebikes', con=engine, index=False)

252